# Analysis: Single Match Relative EV (Value Bet Strategy)

This notebook implements a value-betting strategy for MonPetitProno.

## The Edge

The core market inefficiency is that MPP **freezes the points** for each outcome (`g1, g2, g3`) long before the match, based on old bookmaker odds. The **live bookmaker odds** (`c1, c2, c3`) give us a much more accurate "ground truth" probability (`gtp1, gtp2, gtp3`) of each outcome.

Our edge comes from finding bets where the **frozen points are high relative to the true probability** (`gtp * g` is high).

## The Goal

We don't just want a high EV; we want a high **Relative EV**. We want to maximize our expected points *relative to the other players in the pool*.

**Strategy:** Calculate `Relative EV = E[My_Score] - E[Average_Opponent_Score]` for each of the three possible bets (Bet Win, Bet Draw, Bet Loss) and choose the highest one.

In [1]:
import numpy as np
from scipy.stats import binom
import sys
import os

# Add the parent directory to the path to import our custom module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from mpp_project.core import calculate_true_outcome_probas_from_odds, get_simple_ev, get_ev_with_bonus

In [2]:
# --- Configuration ---

# MPP frozen points for each outcome (Win, Draw, Loss)
# g1, g2, g3
mpp_points = np.array([32, 143, 179])

# CURRENT Bookmaker odds (our "ground truth")
# c1, c2, c3
bookmaker_odds = np.array([3.3, 2.85, 2.6])

# Assumed repartition of bets from other players
opp_bet_repartition = np.array([0.26, 0.30, 1 - 0.26 - 0.30])

# --- Perfect score probabilities ---
# Here, we assume :
#     - the player will chose the best perfect score options ;
#     - the oponents will, on average, chose the second best perfect score.
# Player's proba of perfect score, given a correct outcome (typically bookmaker probas for 1-0, 0-0/1-1 and 0-1)
player_perfect_probas = np.array([0.24, 0.14, 0.06])

# Opponent's proba of perfect score, given a correct outcome (typically bookmaker probas for 2-1, 1-1/2-2 and 1-2)
opp_perfect_probas = np.array([0.135, 0.06, 0.03])


In [3]:
# --- 1. Calculate "Ground Truth" Probabilities ---

# Use our core function to get the "true" probas from current odds
# gtp1, gtp2, gtp3
true_outcome_probas = calculate_true_outcome_probas_from_odds(bookmaker_odds)

print("--- Ground Truth Probabilities (from Current Odds) ---")
print(f"P(Win):  {true_outcome_probas[0]:.3f}")
print(f"P(Draw): {true_outcome_probas[1]:.3f}")
print(f"P(Loss): {true_outcome_probas[2]:.3f}")

--- Ground Truth Probabilities (from Current Odds) ---
P(Win):  0.292
P(Draw): 0.338
P(Loss): 0.370


In [9]:
# --- Intermediate Interesting Results : Simple EV ---

print("\n--- EV (based on ground truth probabilities, no bonus) ---")
simple_ev_1 = get_simple_ev(true_outcome_probas[0], mpp_points[0])
simple_ev_2 = get_simple_ev(true_outcome_probas[1], mpp_points[1])
simple_ev_3 = get_simple_ev(true_outcome_probas[2], mpp_points[2])
print(f"EV (Win, Draw, Loss):  {simple_ev_1:.1f}, {simple_ev_2:.1f}, {simple_ev_3:.1f}")


--- EV (based on ground truth probabilities, no bonus) ---
EV (Win, Draw, Loss):  9.3, 48.3, 66.3


In [11]:
# --- 2. Calculate Expected Relative Scores ---
# E[Rel_EV] = E[My_Score] - E[Average_Opponent_Score]

# --- Pre-calculate Expected Points (if correct, with bonus) ---
# E[Points | Correct] = G * (1 + P(perfect | Correct))

# My expected points if I am correct
exp_gain_self = mpp_points * (1 + player_perfect_probas)

# Opponents' expected points if they are correct
exp_gain_opp = mpp_points * (1 + opp_perfect_probas)

print("--- Expected Gain (if correct, with bonus) ---")
print(f"Self (Win, Draw, Loss):  {exp_gain_self[0]:.1f}, {exp_gain_self[1]:.1f}, {exp_gain_self[2]:.1f}")
print(f"Opp  (Win, Draw, Loss):  {exp_gain_opp[0] :.1f}, {exp_gain_opp[1] :.1f}, {exp_gain_opp[2] :.1f}")

--- Expected Gain (if correct, with bonus) ---
Self (Win, Draw, Loss):  39.7, 163.0, 189.7
Opp  (Win, Draw, Loss):  36.3, 151.6, 184.4


In [12]:
# --- 3. Calculate E[Average_Opponent_Score] (Pool Average) ---
# This is the "background" score of the pool, constant regardless of my bet.

# E[Opp_Score] = P(Win)*E[Opp_Score|Win] + P(Draw)*E[Opp_Score|Draw] + P(Loss)*E[Opp_Score|Loss]
# We use the "true" probabilities (gtp) for P(Win), P(Draw), P(Loss)

# E[Opp_Score|Win] = P(Opp_Bet_Win) * E[Opp_Gain|Win]
# P(Opp_Bet_Win) is the opponent repartition
exp_opp_score_if_win  = opp_bet_repartition[0] * exp_gain_opp[0]
exp_opp_score_if_draw = opp_bet_repartition[1] * exp_gain_opp[1]
exp_opp_score_if_loss = opp_bet_repartition[2] * exp_gain_opp[2]

exp_avg_opp_score = (true_outcome_probas[0] * exp_opp_score_if_win + 
                     true_outcome_probas[1] * exp_opp_score_if_draw + 
                     true_outcome_probas[2] * exp_opp_score_if_loss)

print("--- Expected Avg Opponent Score (Pool Average) ---")
print(f"E[Avg_Opp_Score]: {exp_avg_opp_score:.2f}")

--- Expected Avg Opponent Score (Pool Average) ---
E[Avg_Opp_Score]: 48.16


In [13]:
# --- 4. Calculate E[My_Score] for each of my 3 bet choices ---

# E[My_Score_Bet_Win] = P(Win) * E[My_Gain|Win] = gtp1 * exp_gain_self[0]
exp_my_score_bet_1 = true_outcome_probas[0] * exp_gain_self[0]
exp_my_score_bet_2 = true_outcome_probas[1] * exp_gain_self[1]
exp_my_score_bet_3 = true_outcome_probas[2] * exp_gain_self[2]

print("--- My Expected Score (for each bet choice) ---")
print(f"E[My_Score | Bet Win]:  {exp_my_score_bet_1:.2f}")
print(f"E[My_Score | Bet Draw]: {exp_my_score_bet_2:.2f}")
print(f"E[My_Score | Bet Loss]: {exp_my_score_bet_3:.2f}")

# --- 5. Calculate Final Relative EV ---
# E[Relative_EV] = E[My_Score] - E[Average_Opponent_Score]

final_rel_ev_bet_1 = exp_my_score_bet_1 - exp_avg_opp_score
final_rel_ev_bet_2 = exp_my_score_bet_2 - exp_avg_opp_score
final_rel_ev_bet_3 = exp_my_score_bet_3 - exp_avg_opp_score

print("\n--- Final Relative EV (E[My_Score] - E[Avg_Opp_Score]) ---")
print(f"Rel. EV (Bet Win):  {final_rel_ev_bet_1:.2f}")
print(f"Rel. EV (Bet Draw): {final_rel_ev_bet_2:.2f}")
print(f"Rel. EV (Bet Loss): {final_rel_ev_bet_3:.2f}")

--- My Expected Score (for each bet choice) ---
E[My_Score | Bet Win]:  11.58
E[My_Score | Bet Draw]: 55.08
E[My_Score | Bet Loss]: 70.27

--- Final Relative EV (E[My_Score] - E[Avg_Opp_Score]) ---
Rel. EV (Bet Win):  -36.58
Rel. EV (Bet Draw): 6.92
Rel. EV (Bet Loss): 22.11
